In [ ]:
from adam.casadi.computations import KinDynComputations
from adam.core import link_parametric
from adam.geometry import utils
import numpy as np
import casadi as cs

In [ ]:
def SX2DM(sx):
    return cs.DM(sx)

## Import the robot .urdf


In [ ]:
urdf_path = "/home/carlotta/iit_ws/ergocub-gazebo-simulations/models/stickBot/model.urdf"
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
# Specify the root link
root_link = 'root_link'
link_name_list = ['l_lower_leg']
link_name_list_empty  = []
r_hip_r_char = link_parametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = link_parametric.JointCharacteristics(-0.055989)
link_characteristics = {link_name_list[0]: r_hip_r_char}
joint_characteristics = {'l_ankle_pitch': r_hip_yaw_char}
kinDyn = KinDynComputations(urdf_path, joints_name_list, root_link, link_name_list, link_characteristics, joint_characteristics)


In [ ]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_Pos_RPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
density = np.random.rand(len(link_name_list))
lenght_multiplier = np.random.rand(len(link_name_list))

## Mass Matrix 

In [ ]:
M = kinDyn.mass_matrix_fun()
print('Mass matrix:\n', SX2DM(M(H_b, s, density, lenght_multiplier)))

## Centroidal Momentum Matrix 

In [ ]:
Jcm = kinDyn.centroidal_momentum_matrix_fun()
print('Centroidal Momentum Matrix:\n', SX2DM(Jcm(H_b, s, density, lenght_multiplier)))

## Center of mass position

In [ ]:
CoM = kinDyn.CoM_position_fun()
print('Center of Mass position:\n', SX2DM(CoM(H_b, s, density, lenght_multiplier)))

## Total Mass

In [ ]:
total_mass = kinDyn.get_total_mass()
print('Total mass:\n', SX2DM(total_mass(density, lenght_multiplier)))

## Jacobian 

In [ ]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian of the left sole:\n', SX2DM(J(H_b, s, density, lenght_multiplier)))

## Relative jacobian 

In [ ]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian between the root link and left sole:\n', SX2DM(J_r(s, density, lenght_multiplier)))

## Forward kinematics 

In [ ]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('Forward kinematics to the left sole:\n', SX2DM(H(H_b, s, density, lenght_multiplier)))

## Bias force term 

In [ ]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

## Coriolis term

In [ ]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

## Gravity term 

In [ ]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, density, lenght_multiplier)))